In [1]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
from pathlib import Path
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score
from scipy.special import softmax as softmax
import matplotlib.pyplot as plt

from dataloader.builder import build_dataset
from experiment_setup import build_estimator
from uncertainty_estimator.masks import build_masks, DEFAULT_MASKS
from analysis.metrics import uq_ndcg

from model.cnn import SimpleConv, MediumConv, StrongConv
from model.trainer import Trainer, EnsembleTrainer

# torch.cuda.set_device(1)
#

In [3]:
config = {
    'use_cuda': True,
    'seed': 1,
    
    'nn_runs': 150,
    'patience': 5,
    'dropout_uq': 0.5,
    
    'n_models': 10, 
    
    # 'dataset': 'mnist',
    'dataset': 'cifar_10',
   
    'model_class': StrongConv,
    'train_samples': 45_000,
    'epochs': 50,
    'batch_size': 256,
    'log_interval': 150,
    'lr': 1e-2,
    'num_classes': 9
}
restore_model = True


In [4]:
dataset = build_dataset(config['dataset'], val_size=10_000)
x_train, y_train = dataset.dataset('train')
x_val, y_val = dataset.dataset('val')

def cut_class(x, y, class_num):
    new_x = x[np.where(y!=class_num)]
    new_y = y[np.where(y!=class_num)]
    ood = x[np.where(y==class_num)]
    return new_x, new_y, ood

x_train, y_train, x_ood = cut_class(x_train, y_train, '9')
x_val, y_val, _ = cut_class(x_val, y_val, '9')



In [5]:
def scale(images):
    return (images - 128) / 128
x_train = scale(x_train)
x_val = scale(x_val)
x_ood = scale(x_ood)

In [6]:
if config['dataset'] == 'mnist':
    input_shape = (-1, 1, 28, 28)
elif config['dataset'] == 'cifar_10':
    input_shape = (-1, 3, 32, 32)
x_train = x_train.reshape(input_shape)
x_val = x_val.reshape(input_shape)
x_ood = x_ood.reshape(input_shape)

y_train = y_train.astype('long').reshape(-1)
y_val = y_val.astype('long').reshape(-1)



In [7]:
train_samples = config['train_samples']
epochs = config['epochs']
val_samples = 2000
patience = config['patience']

idxs = np.random.choice(len(x_train), train_samples, replace=False)
train_set = (x_train[idxs], y_train[idxs])

idxs = np.random.choice(len(x_val), val_samples, replace=False)
val_set = (x_val[idxs], y_val[idxs]) 
print(train_set[0].shape)
print(val_set[0].shape)

(45000, 3, 32, 32)
(2000, 3, 32, 32)


In [8]:
model_class = config['model_class'] 
model = model_class(config['num_classes'])

model_dir = Path('experiments') / 'data'
model_path = model_dir / f"model_{config['dataset']}_{config['model_class'].__name__}.pt"

if restore_model and os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path))
    trainer = Trainer(model)
else:
    trainer = Trainer(model)
    trainer.fit(
        train_set, val_set , epochs=epochs, verbose=True, patience=patience)
    torch.save(model.state_dict(), model_path)

    

In [9]:
trainer.predict(val_set[0][:5])

array([[5],
       [1],
       [6],
       [2],
       [4]])

array([7, 1, 6, 4, 4])